# بسم الله الرحمن الرحیم

# محمد مهدی شفیقی - پروژه نهایی درس مباحث ویژه

# تحلیل شبکه همکاری علمی CA-HepTh

این پروژه به تحلیل ویژگی‌های ساختاری و دینامیکی شبکه هم‌نویسندگی در حوزه فیزیک انرژی بالا می‌پردازد.

In [ ]:
# فقط در صورت نیاز اجرا کن
!pip install --quiet networkx pandas numpy matplotlib python-louvain tqdm scipy
# برای عملکرد بهتر (اختیاری، سریع‌تر و مقیاس‌پذیرتر)
!pip install --quiet python-igraph leidenalg


## تنظیم مسیر داده — مسیر را طبق محل واقعی تغییر بده


In [2]:
DATA_FOLDER = "../data/cit-HepTh-abstracts"   # محتوای پوشه: 1992/, 1993/, ...


# ایمپورت‌های پایه


In [3]:
import os, re, gc, pickle, math, time
from itertools import combinations
from collections import defaultdict
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# توابع مفید


In [4]:
def save_pickle(obj, path):
    with open(path, "wb") as f:
        pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)

def load_pickle(path):
    with open(path, "rb") as f:
        return pickle.load(f)


# چک محیط و نمونه‌برداری فایل‌ها + تابع استخراج نویسنده (اجرای اولیه)

In [6]:
# A-0: Environment check + sample parsing of .abs files
# اجرا در یک سلول جدید در Jupyter notebook

import os, re, time
from collections import defaultdict
from itertools import combinations
try:
    from tqdm.auto import tqdm
except Exception:
    # tqdm اختیاری است؛ اگر نصب نیست، از آن صرفنظر می‌کنیم
    def tqdm(x, **_): 
        return x

# مسیرِ پوشه‌ی cit-HepTh-abstracts را بر اساس ساختار تو تنظیم کن:
DATA_ROOT = "../data/cit-HepTh-abstracts"   # اگر مسیرت متفاوت است این را تغییر بده

# 1) شمارش فایل‌ها و نام پوشه‌ها (سال‌ها)
years = sorted([d for d in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, d))])
print("Found year-folders (sample):", years[:6], " ... total:", len(years))

year_file_counts = {}
total_files = 0
for y in years:
    p = os.path.join(DATA_ROOT, y)
    files = [f for f in os.listdir(p) if f.endswith('.abs')]
    year_file_counts[y] = len(files)
    total_files += len(files)

print(f"Total .abs files: {total_files}")
print("Files per year (first 10):")
for y in years[:10]:
    print(f"  {y}: {year_file_counts[y]} files")

# 2) Robust author-extraction function (tries چند الگو)
_author_patterns = [
    re.compile(r'Authors:\s*(.*?)\n(?:Title:|Comments:|\\\n|$)', re.DOTALL | re.IGNORECASE),
    re.compile(r'Authors:\s*(.*)', re.IGNORECASE),
]

def extract_authors_from_text(txt):
    """
    تلاش می کند رشته‌ی نویسندگان را از متن استخراج کند.
    بازگشتی: لیست اسامی نویسنده‌ها (هر اسم تمیزشده)
    """
    # بعضی فایل‌ها از backslash \\ برای جدا کردن بخش‌ها استفاده می‌کنند، آن‌ها را به newline تبدیل کن
    t = txt.replace('\\\n', '\n').replace('\\', '\n')
    authors_text = None
    for pat in _author_patterns:
        m = pat.search(t)
        if m:
            authors_text = m.group(1)
            break
    if not authors_text:
        # fallback: خطی جستجو کن
        for line in t.splitlines():
            if line.strip().lower().startswith("authors:"):
                authors_text = line.split(':',1)[1]
                break
    if not authors_text:
        return []  # هیچ نویسنده‌ای پیدا نشد

    # پاک‌سازی و جداسازی: "and" و کاما و ; را مدنظر قرار بده
    authors_text = authors_text.replace('\n', ' ')
    # بعضی فرمت‌ها "A and B" دارند
    authors_text = re.sub(r'\sand\s', ',', authors_text)
    # جداکننده‌ها: ',' یا ';' یا ' and '
    parts = re.split(r',|;|\band\b', authors_text)
    authors = []
    for p in parts:
        p = p.strip()
        if not p:
            continue
        # حذف موارد غیرِ اسم (مثل affiliation داخل پرانتز)
        p = re.sub(r'\(.*?\)', '', p).strip()
        # normalize spaces
        p = re.sub(r'\s+', ' ', p)
        authors.append(p)
    # unique preserving order
    seen = set()
    authors_clean = []
    for a in authors:
        key = a.lower()
        if key not in seen:
            seen.add(key)
            authors_clean.append(a)
    return authors_clean

# 3) نمایش چند نمونه فایل و نتیجه‌ی استخراج نویسنده
SAMPLES_TO_SHOW = 6
sample_files = []
for y in years:
    d = os.path.join(DATA_ROOT, y)
    files = [f for f in os.listdir(d) if f.endswith('.abs')]
    if files:
        sample_files.append((y, files[:1][0], os.path.join(d, files[0])))
    if len(sample_files) >= SAMPLES_TO_SHOW:
        break

print("\nSample files and extracted authors:")
for y, fname, fpath in sample_files:
    try:
        with open(fpath, 'r', encoding='utf-8', errors='ignore') as fh:
            txt = fh.read(4000)  # فقط 4k اول برای نمایش
    except Exception as e:
        txt = f"(error reading: {e})"
    authors = extract_authors_from_text(txt)
    print(f"\nYear {y} | file: {fname}")
    print(" Extracted authors:", authors[:10])

# 4) (اختیاری) تست زمان خواندن تعداد نمونه‌ای از فایل‌ها
NTEST = 200   # تعداد فایل‌ها که برای سنجش سرعت پردازش بررسی می‌کنیم
t0 = time.time()
count = 0
for y in years:
    d = os.path.join(DATA_ROOT, y)
    for fn in os.listdir(d):
        if not fn.endswith('.abs'):
            continue
        count += 1
        if count > NTEST:
            break
    if count > NTEST:
        break
t1 = time.time()
print(f"\nRough IO check: enumerated ~{min(count, NTEST)} files in {t1-t0:.2f}s")

print("\nA-0 done. If sample author extraction looks reasonable, reply with 'A-0 OK' and we'll run A-1: build efficient temporal-edge lists (ids + per-year CSV dump).")


Found year-folders (sample): ['1992', '1993', '1994', '1995', '1996', '1997']  ... total: 12
Total .abs files: 29555
Files per year (first 10):
  1992: 1367 files
  1993: 2058 files
  1994: 2377 files
  1995: 2303 files
  1996: 2606 files
  1997: 2673 files
  1998: 2758 files
  1999: 2803 files
  2000: 3126 files
  2001: 3153 files

Sample files and extracted authors:

Year 1992 | file: 9201001.abs
 Extracted authors: ['C. Itzykson', 'J.-B. Zuber']

Year 1993 | file: 9301001.abs
 Extracted authors: ['G.K.Savvidy', 'K.G.Savvidy']

Year 1994 | file: 9401001.abs
 Extracted authors: ['Jorge Ananias Neto']

Year 1995 | file: 9501001.abs
 Extracted authors: []

Year 1996 | file: 9601001.abs
 Extracted authors: []

Year 1997 | file: 9701001.abs
 Extracted authors: ['M. Zyskin We consider d-dimensional Riemanian manifolds which admit d-2 commuting space-like Killing vector fields', 'orthogonal to a surface', 'containing two one-parametric families of light-like curves. The condition of the Ric